In [146]:
import pandas as pd
import numpy as np
#from bs4 import BeautifulSoup
import re
from collections import Counter

In [147]:
df = pd.read_csv("data/train.tsv", sep="\t", index_col=0)

In [148]:
df.head()

,true
0,с конец 1811 го год начаться усиленный вооруже...
1,12 июнь сила западный европа переслать граница...
2,миллион человек совершать друг против друг так...
3,что произвести это необычайный событие
4,какой быть причина он


In [149]:
BOS = "BOS"
EOS = "EOS"
UNK = "UNK"

def prepare_sentences(sentences, word_threshold=2, stage_train=True):
    print(type(sentences[0]), re.split("\W+", sentences[0]))
    
    # оставляем только alphanumeric
    clean_sentences = [re.split("\W+", str(s)) for s in sentences]
    
    # заменяем числа на NUM
    clean_sentences = [[w.replace("\d+", "NUM") for w in s if w] for s in clean_sentences]
    
    # вводим тег UNKNOWN: UNK
    if stage_train:
        counter = Counter()

        for s in clean_sentences:
            for w in s:
                counter[w] += 1
    
        print("Filtered out word types :", len([w for w in counter if counter[w] <= word_threshold]))
        print("Filtered out words count:", sum([counter[w] for w in counter if counter[w] <= word_threshold]))
    
        # выкидываем редкие, и заменяем их на специальный тег
        clean_sentences = [[w if counter[w] > word_threshold else UNK for w in s] for s in clean_sentences]            
    
    word2index = { BOS: 0, EOS: 1, UNK: 2}
    index2word = { 0: BOS, 1: EOS, 2: UNK}
    
    counter = max(word2index.values()) + 1

    for s in clean_sentences:
        for w in s:
            if not w in word2index:
                word2index[w] = counter
                index2word[counter] = w
                counter += 1
                
    return word2index, index2word, clean_sentences

In [150]:
word2index, index2word, clean_sentences = prepare_sentences(df["true"].tolist())

<class 'str'> ['с', 'конец', '1811', 'го', 'год', 'начаться', 'усиленный', 'вооружение', 'и', 'сосредоточение', 'сила', 'западный', 'европа', 'и', 'в', '1812', 'год', 'сила', 'этот', 'миллион', 'человек', 'считать', 'тот', 'который', 'перевозить', 'и', 'кормить', 'армия', 'двинуться', 'с', 'запад', 'на', 'восток', 'к', 'граница', 'россия', 'к', 'который', 'точно', 'так', 'же', 'с', '1811', 'го', 'год', 'стягиваться', 'сила', 'россия']
Filtered out word types : 8565
Filtered out words count: 10787


In [151]:
clean_sentences[:10]

[['с',
  'конец',
  '1811',
  'го',
  'год',
  'начаться',
  'усиленный',
  'вооружение',
  'и',
  'UNK',
  'сила',
  'западный',
  'европа',
  'и',
  'в',
  '1812',
  'год',
  'сила',
  'этот',
  'миллион',
  'человек',
  'считать',
  'тот',
  'который',
  'UNK',
  'и',
  'кормить',
  'армия',
  'двинуться',
  'с',
  'запад',
  'на',
  'восток',
  'к',
  'граница',
  'россия',
  'к',
  'который',
  'точно',
  'так',
  'же',
  'с',
  '1811',
  'го',
  'год',
  'UNK',
  'сила',
  'россия'],
 ['12',
  'июнь',
  'сила',
  'западный',
  'европа',
  'переслать',
  'граница',
  'россия',
  'и',
  'начаться',
  'война',
  'то',
  'есть',
  'совершиться',
  'противный',
  'человеческий',
  'разум',
  'и',
  'весь',
  'человеческий',
  'природа',
  'событие'],
 ['миллион',
  'человек',
  'совершать',
  'друг',
  'против',
  'друг',
  'такой',
  'бесчисленный',
  'количество',
  'злодеяние',
  'обман',
  'измена',
  'воровство',
  'UNK',
  'и',
  'UNK',
  'фальшивый',
  'ассигнация',
  'грабёж',

In [152]:
print("Total number of sentences :\t", len(clean_sentences))
print("Total number of words     :\t", sum([len(sent) for sent in clean_sentences]))
print("Total number of word types:\t", len(set([w for sent in clean_sentences for w in sent])))

Total number of sentences :	 14996
Total number of words     :	 228126
Total number of word types:	 6409


In [153]:
def augment(sentence, context_size):
    """
        Добиваем символы начала и конца строки к каждому предложению
    """
    return [BOS] * context_size + sentence + [EOS] * context_size

def enumerate_sentences(clean_sentences, context_size, word2index):
    """
        Добиваем символами начала и конца и конвертируем слова в индексы
    """

    contexts = []
    targets = []
    UNK_id = word2index[UNK]

    for sentence in clean_sentences:

        aligned_sentence =  augment(sentence, context_size) 

        for i in range(context_size, len(sentence) - context_size, 1):
            
            # берём предшествующий контекст
            context = aligned_sentence[i - context_size:i]
            context = [word2index[c] if c in word2index else UNK_id for c in context]
            target = word2index[aligned_sentence[i]] if aligned_sentence[i] in word2index else UNK_id
            
            contexts.append(context)
            targets.append(target)
    
    return contexts, targets

In [154]:
def chunks(l0, l1, n):
    
    assert len(l0) == len(l1)
    coll0, coll1 = [], []
    
    for i in range(0, len(l0), n):
        coll0.append(l0[i:i + n])
        coll1.append(l1[i:i + n])
        
    return coll0, coll1

In [155]:
CONTEXT_SIZE = 3
BATCH_SIZE = 2048

# строим контексты и цели
contexts, targets = enumerate_sentences(clean_sentences, CONTEXT_SIZE, word2index)

In [156]:
contexts[:10]

[[0, 0, 0],
 [0, 0, 3],
 [0, 3, 4],
 [3, 4, 5],
 [4, 5, 6],
 [5, 6, 7],
 [6, 7, 8],
 [7, 8, 9],
 [8, 9, 10],
 [9, 10, 11]]

In [157]:
batches = list(zip(contexts, targets))

In [158]:
batches[:10]

[([0, 0, 0], 3),
 ([0, 0, 3], 4),
 ([0, 3, 4], 5),
 ([3, 4, 5], 6),
 ([4, 5, 6], 7),
 ([5, 6, 7], 8),
 ([6, 7, 8], 9),
 ([7, 8, 9], 10),
 ([8, 9, 10], 11),
 ([9, 10, 11], 2)]

In [159]:
from collections import defaultdict
from tqdm import tqdm_notebook
from functools import lru_cache

class NGramFreqsLanguageModeler(object):
    
    def __init__(self, vocab_size, context_size):
        # не обязательная часть, насколько я понимаю
        # super(NGramFreqsLanguageModeler, self).__init__()
        self.vocab_size = vocab_size
        self.context_size = context_size
        # вероятности всех слов после этого контекста
        self.ngram_dict = defaultdict(lambda: defaultdict(int))
        # То же самое, что ngram_dict, только длина контекста = 2. Зачем?
        self.n_1_gram_dict = defaultdict(lambda: defaultdict(int))
        # количество раз сколько встречается каждый контекст
        self.contexts_counts = defaultdict(int)
        # коэффициент лапласовского сглаживания
        self.eps = .1
    
    def fit(self, contexts, targets):
        
        self.contexts_counts = defaultdict(int)
        
        for c, t in zip(contexts, targets):
            c = tuple(c)
            self.ngram_dict[c][t] += 1
            self.contexts_counts[c] += 1
            
            # намёк!
#             self.n_1_gram_dict[c[1:]][t] += 1

            
        print("Total n-1 grams", len(self.ngram_dict), list(self.ngram_dict)[:10])
        
        # нормализуем частоты
        for c in tqdm_notebook(self.ngram_dict.keys()):
            for t in self.ngram_dict[c]:
                # нормируем слово в контексте на количество таких контекстов и общее количество слов
                self.ngram_dict[c][t] = (self.ngram_dict[c][t] +  self.eps) / \
                                            (self.contexts_counts[c] + self.vocab_size * self.eps)
        
    @lru_cache(1000000)
    def prob_dist(self, input_context):
        """
            Takes ngram as a tuple
        """
        
        probs = np.zeros(self.vocab_size) + \
                    self.eps / (self.vocab_size * self.eps + self.contexts_counts[input_context])
        
        counts = self.ngram_dict[input_context]
        
        # если есть хоть какие-то счётчики
        if counts:
            # проставим осмысленные частоты
            for target, freq in counts.items():
                probs[target] = freq
                
        return probs

In [160]:
simple_model = NGramFreqsLanguageModeler(context_size=CONTEXT_SIZE, vocab_size=len(word2index))
simple_model.fit(contexts, targets)

Total n-1 grams 112596 [(0, 0, 0), (0, 0, 3), (0, 3, 4), (3, 4, 5), (4, 5, 6), (5, 6, 7), (6, 7, 8), (7, 8, 9), (8, 9, 10), (9, 10, 11)]


In [161]:
test = "BOS"
prepared_text = augment(prepare_sentences(test, stage_train=False)[2][0], CONTEXT_SIZE)[-CONTEXT_SIZE:]
prepared_text

<class 'str'> ['B']


['EOS', 'EOS', 'EOS']

In [162]:
for i in range(CONTEXT_SIZE, 10 + CONTEXT_SIZE):
    
    idx = [word2index[w] for w in prepared_text[:i]]    
    
    predict = simple_model.prob_dist(tuple(idx[-CONTEXT_SIZE:])) 
    
#     predict = predict - predict.min()  
#     predict /= sum(predict)
    
    selected_word = np.random.choice(a=range(len(word2index)), p=predict)    
    prepared_text.append(index2word[selected_word])
    
    print("Генерация:", " ".join(prepared_text[CONTEXT_SIZE - 1:]))

Генерация: EOS feu
Генерация: EOS feu привыкнуть
Генерация: EOS feu привыкнуть владыка
Генерация: EOS feu привыкнуть владыка вникать
Генерация: EOS feu привыкнуть владыка вникать посердиться
Генерация: EOS feu привыкнуть владыка вникать посердиться решение
Генерация: EOS feu привыкнуть владыка вникать посердиться решение жёсткий
Генерация: EOS feu привыкнуть владыка вникать посердиться решение жёсткий верхом
Генерация: EOS feu привыкнуть владыка вникать посердиться решение жёсткий верхом открываться
Генерация: EOS feu привыкнуть владыка вникать посердиться решение жёсткий верхом открываться христос


In [163]:
df_test = pd.read_csv("data/task.tsv", sep="\t", index_col=0)

In [164]:
df_test.shape

(1666, 2)

In [165]:
df_test.head()

,text1,text2
0,что есть власть,что власть есть
1,власть есть совокупность воля перенести на оди...,лицо воля один перенести есть власть совокупно...
2,какой переноситься при на воля условие лицо ма...,при какой условие переноситься воля масса на о...
3,при условие выражение лицо воля весь человек,лицо воля выражение человек весь условие при
4,власть то есть есть власть,то есть власть есть власть


In [166]:
def wseq2indexes(wseq):
    return tuple([word2index[w] if (w in word2index) else word2index["UNK"] for w in wseq])

In [167]:
len(prepare_sentences(df_test[text_n].tolist())[2])

<class 'str'> ['что', 'власть', 'есть']
Filtered out word types : 2958
Filtered out words count: 3667


1666

In [168]:
text_probs = {}

for text_n in ["text1", "text2"]:
    text_probs[text_n] = defaultdict(int)
    
    for index, sent in enumerate(prepare_sentences(df_test[text_n].tolist())[2]):
#         print("\nTEXT")
        sent = augment(sent, CONTEXT_SIZE)
#         print(sent)
        for i in range(len(sent) - CONTEXT_SIZE):
            context = sent[i: i + CONTEXT_SIZE]
            target = sent[i + CONTEXT_SIZE]

            context_indexes = wseq2indexes(sent[i: i + CONTEXT_SIZE])

            target_index = word2index.get(target, word2index["UNK"])
#             print(context_indexes, target_index)

            prob = simple_model.ngram_dict[context_indexes][target_index]
#             print(prob)
            text_probs[text_n][index] += prob

#             print(context, target, prob)

<class 'str'> ['что', 'есть', 'власть']
Filtered out word types : 2958
Filtered out words count: 3667
<class 'str'> ['что', 'власть', 'есть']
Filtered out word types : 2958
Filtered out words count: 3667


In [169]:
len(text_probs["text2"])

1666

In [170]:
len(text_probs["text2"])

1666

In [171]:
answers = {}
for i in text_probs["text1"]:
    answers[i] = 1 if text_probs["text1"][i] >= text_probs["text2"][i] else 0

In [172]:
len(text_probs["text1"])

1666

In [173]:
import csv

with open("data/submission_window3.csv", "wt") as f:
    writer = csv.writer(f) 
    #writer.writerow(["id", "which"])
    for i, answer in answers.items():
        writer.writerow([i, answer])

In [174]:
from sklearn.metrics import accuracy_score

In [175]:
y_true = pd.read_csv("data/true_labels.tsv")

In [176]:
y_pred = pd.read_csv("data/submission_window3.csv")

In [177]:
y_true.shape, y_pred.shape

((1665, 1), (1665, 2))

In [178]:
accuracy_score(y_true.iloc[:, 0].values, y_pred.iloc[:, 1].values)

0.6312312312312313